In [ ]:
import skimage
import cv2
from skimage import feature
from skimage import io
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.color import rgb2gray
from PIL import Image
from skimage.feature import canny
from skimage import exposure
from skimage.filters import try_all_threshold, sobel
from skimage.filters import gaussian
from skimage.morphology import binary_dilation, binary_erosion
from skimage.transform import rescale, resize, downscale_local_mean, estimate_transform

from skimage.draw import line
from skimage import data
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.viewer import ImageViewer
from skimage import filters, segmentation, io
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import sys, os

import warnings
warnings.filterwarnings('ignore')

## Convert color image to numpy.ndarray and to grayscale

In [ ]:
file_path = './hw_im_1/IMG_20190312_183838.jpg'
image = io.imread(file_path)
image = resize(image, (image.shape[0] // 4, image.shape[1] // 4),
                       anti_aliasing=True)
image_bw = rgb2gray(image)
plt.imshow(image_bw,cmap="gray")
H = image_bw.shape[0]
W = image_bw.shape[1]
H,W


## Improve exposure

In [ ]:
gamma_corrected = exposure.adjust_gamma(image_bw, 2)
plt.imshow(gamma_corrected,cmap="gray")

## Blur image

In [ ]:
blurred_image = gaussian(gamma_corrected, multichannel=True, sigma=(3,3))
plt.imshow(blurred_image,cmap="gray")

## Find best thresholding function

In [ ]:
fig, ax = try_all_threshold(blurred_image, figsize=(10, 8), verbose=False)
plt.show()

In [ ]:
# Import the skimage threshold_otsu package
from skimage.filters import threshold_otsu
# get the global optimal threshold for the leopard image
globalthreshold = threshold_otsu(image=image_bw)
# apply the threshold to the gray image to obtain a binary image
thresholded =image_bw > globalthreshold
# finally show the binary image
plt.imshow(thresholded,cmap="gray")

In [ ]:
edgedet = canny(thresholded)
imgplot = plt.imshow(edgedet, cmap="gray")

## Detect edges using Canny algorithm

In [ ]:
image_edgedet = canny(thresholded, sigma=0)
imgplot = plt.imshow(image_edgedet,cmap="gray")

In [ ]:
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
h, theta, d = hough_line(image_edgedet, theta=tested_angles);

In [ ]:
# Constructing test image
image = binary_dilation(image_edgedet)

tested_angles = np.linspace(-np.pi/2, np.pi/2, 360, endpoint=False)
h, theta, d = hough_line(image, theta=tested_angles)

# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

angle_step = 0.5 * np.diff(theta).mean()
d_step = 0.5 * np.diff(d).mean()
bounds = [np.rad2deg(theta[0] - angle_step),
          np.rad2deg(theta[-1] + angle_step),
          d[-1] + d_step, d[0] - d_step]
ax[1].imshow(np.log(1 + h), extent=bounds, cmap=cm.gray, aspect=1 / 1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image, cmap=cm.gray)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

line_params = []
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    x0, y0 = dist * np.array([np.cos(angle), np.sin(angle)])
    ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))
    line_params.append((x0,y0,np.tan(angle + np.pi/2)))

plt.tight_layout()
plt.show()

## Conversion from one mathematical notion to the other

In [ ]:
def convert_to_homogenous(line_params):
    homogenous_forms = []
    for param in line_params:
    #convert to directional form of the linear function
    # y = ax + b
        x = param[0]
        y = param[1]
        a = param[2]
        b = y - a*x
    #convert to cannonical form of the linear function
    # Ax + By + C = 0
        A = -a
        B = 1
        C = -b
        homogenous_forms.append([A,B,C])
        
    return homogenous_forms

def find_intersections(homogenous_forms):
    # print(homogenous_forms)
    intersections = set()
    for line_1 in homogenous_forms:
        for line_2 in homogenous_forms:
            #Remove lines that are too close
            print(line_2[0])
            e = np.cross(line_1,line_2)
            #Equation has no solution, cannot divide by zero
            if e[2] == 0:
                continue
            else:
                x = e[0]/e[2]
                y = e[1]/e[2]
                # Intersection points are part of the image
                if (0<= x<= W) and (0 <= y <= H):
                    intersections.add((x,y))

    intersections = list(intersections)
    return intersections




In [ ]:
%%capture
homogenous_forms = convert_to_homogenous(line_params)
intersections = find_intersections(homogenous_forms)

In [ ]:
alphabet = ['A','B','C','D']
for c in intersections:
    # plt.plot(c[0], c[1],'ob',color="green")
    plt.text(c[0], c[1],f'{alphabet.pop(0)}', color='blue')
plt.imshow(image_bw, cmap=cm.gray)
plt.show()

In [ ]:
#A4 paper card = 210 x 297 mm
# 3 pixels per mm
w,h = 630, 891

A = np.array([w,h])
B = np.array([0,h])
C = np.array([0,0])
D = np.array([w,0])

In [ ]:
a = intersections[0]
b = intersections[1]
c = intersections[2]
d = intersections[3]

a = np.array([a[0],a[1]])
b = np.array([b[0],b[1]])
c = np.array([c[0],c[1]])
d = np.array([d[0],d[1]])

invtf = estimate_transform("projective",src=np.vstack((A,B,C,D)),
dst=np.vstack((a,b,c,d)))

In [ ]:
im = skimage.transform.warp(image=thresholded,inverse_map=invtf,output_shape=(h,w))
plt.imshow(im,cmap='gray')

## Segment image and crop images based on bounding box coordinates

In [ ]:
row_vals = list([sum(r) for r in im  ])
col_vals = list([sum(c) for c in im.T])

plt.plot(col_vals)
plt.show()

plt.plot(row_vals)
plt.show()

val = filters.threshold_otsu(im)
mask = im < val

clean_border = segmentation.clear_border(mask)

plt.imshow(clean_border, cmap='gray')
plt.show()

#######################
# Label image regions #
#######################

labeled = label(clean_border)
image_label_overlay = label2rgb(labeled, image=im)

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
ax.imshow(image_label_overlay)
#plt.show()

#########################################
# Draw bounding box around each article #
#########################################

# create array in which to store cropped articles
cropped_images = []

# define amount of padding to add to cropped image
pad = 20

for region_index, region in enumerate(regionprops(labeled)):
  if region.area < 50:
    continue

  # draw a rectangle around the segmented articles
  # bbox describes: min_row, min_col, max_row, max_col
  minr, minc, maxr, maxc = region.bbox
  
  # use those bounding box coordinates to crop the image
  cropped_images.append(im[minr-pad:maxr+pad, minc-pad:maxc+pad])
  
  rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=2)

  ax.add_patch(rect)

plt.show()

###############
# Crop images #
###############

out_dir = "segmented_articles/"
if not os.path.exists(out_dir):
  os.makedirs(out_dir)

# can crop using: cropped = image_array[x1:x2,y1:y2]
for c, cropped_image in enumerate(cropped_images):
  new_image = skimage.transform.resize(cropped_image, (28,28))
  io.imsave( out_dir + str(c) + ".jpeg", new_image)

In [ ]:
images_to_check = []
for i in cropped_images:
    images_to_check.append(skimage.transform.resize(i, (28,28)))

## Training models based on mnist-dataset

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
%%time
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

In [ ]:
for i in images_to_check:
    x = tf.convert_to_tensor(i)
    print(x.shape)
    x = tf.reshape(x,(-1, 28, 28, 1))
    plt.imshow(i,cmap='gray')

    # xtrain = tf.expand_dims(i, axis=-1)
    predictions = model.predict(x)
    print(f'class: {predictions.argmax()}')


## Display results

In [ ]:
rows=3
cols = 2
img_count = 0

fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(14,14))

for i in range(rows):
    for j in range(cols):        
        if img_count < len(images):
            x = tf.convert_to_tensor(images_to_check[img_count])
            x = tf.reshape(x,(-1, 28, 28, 1))
            predictions = model.predict(x)

            axes[i, j].imshow(images_to_check[img_count], cmap='gray')
            axes[i, j].set_title(f'prediction: {predictions.argmax()}')

            img_count+=1